In [1]:
import requests
import csv

from elasticsearch import Elasticsearch
from datetime import datetime

api_key = "f7cd4c26331bfcf236d32f3f23637524"
units = "metric"
lang = "pt_br"
url = "https://api.openweathermap.org/data/2.5/weather"


In [2]:
urls_list = []

with open('coordenadas_capitais_br.csv', ) as coord_csv:
    csv_content = csv.reader(coord_csv)
    next(csv_content)
    for line in csv_content:
        urls_list.append("{}?lat={}&lon={}&appid={}&units={}&lang={}".format(url, line[1], line[2], api_key, units, lang))

urls_list

['https://api.openweathermap.org/data/2.5/weather?lat=-23.5489&lon=-46.6388&appid=f7cd4c26331bfcf236d32f3f23637524&units=metric&lang=pt_br',
 'https://api.openweathermap.org/data/2.5/weather?lat=-23.532511704288144&lon=-46.832017223078914&appid=f7cd4c26331bfcf236d32f3f23637524&units=metric&lang=pt_br',
 'https://api.openweathermap.org/data/2.5/weather?lat=-23.362207765565614&lon=-46.74440645891923&appid=f7cd4c26331bfcf236d32f3f23637524&units=metric&lang=pt_br']

In [3]:
docs_list = []

for url in urls_list:
    req = requests.get(url)
    doc = req.json()
    doc['timestamp'] = datetime.utcfromtimestamp(doc['dt']).isoformat()
    docs_list.append(doc)

docs_list

[{'coord': {'lon': -46.64, 'lat': -23.55},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'céu limpo',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 28.74,
   'feels_like': 29.93,
   'temp_min': 27.72,
   'temp_max': 31.33,
   'pressure': 1018,
   'humidity': 55},
  'visibility': 10000,
  'wind': {'speed': 3.09, 'deg': 80},
  'clouds': {'all': 0},
  'dt': 1699630501,
  'sys': {'type': 1,
   'id': 8394,
   'country': 'BR',
   'sunrise': 1699604152,
   'sunset': 1699651508},
  'timezone': -10800,
  'id': 3448439,
  'name': 'São Paulo',
  'cod': 200,
  'timestamp': '2023-11-10T15:35:01'},
 {'coord': {'lon': -46.832, 'lat': -23.5325},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'céu limpo',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 30.1,
   'feels_like': 34.8,
   'temp_min': 27.66,
   'temp_max': 31.83,
   'pressure': 1017,
   'humidity': 68},
  'visibility': 10000,
  'wind': {'speed': 3.09, 'deg': 80},
  'clouds'

In [4]:
es = Elasticsearch("http://25.63.120.33:9200/")
es

<Elasticsearch(['http://25.63.120.33:9200'])>

In [5]:
for doc in docs_list:
    es_response = es.index(index="teste", document=doc)
    print(es_response['result'])

created
created
created


In [10]:
import re

url = 'https://api.openweathermap.org/data/2.5/weather?lat=-23.5489&lon=-46.6388&appid=f7cd4c26331bfcf236d32f3f23637524&units=metric&lang=pt_br'

lat = re.search(r'lat=([-+]?\d*\.\d+|\d+)', url).group(1)
lon = re.search(r'lon=([-+]?\d*\.\d+|\d+)', url).group(1)

lat, lon

('-23.5489', '-46.6388')